#1. Data Preparation
- Mengunduh dataset MovieLens 100k dari GroupLens.
- Mengimpor data ratings dan movies, lalu menggabungkannya berdasarkan movie_id.
- Membersihkan data dengan memastikan tidak ada nilai kosong yang mengganggu analisis.

In [2]:
#Import Library
import pandas as pd

In [13]:
# Load ratings dan movies
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies = pd.read_csv('u.item', sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL',
                            'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
                            'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

# Gabungkan data ratings dan movies
df = pd.merge(ratings, movies[['movie_id', 'title']], on='movie_id')
df.head()

,user_id,movie_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


#2. Data Analysis
- Menampilkan jumlah total pengguna, jumlah film unik, dan distribusi rating (1–5).
- Mengecek pengguna atau film dengan jumlah rating yang sangat sedikit untuk menghindari data outlier yang bisa mempengaruhi hasil rekomendasi.

In [8]:
# Jumlah pengguna dan film
num_users = df['user_id'].nunique()
num_movies = df['movie_id'].nunique()
print(f"Jumlah pengguna: {num_users}")
print(f"Jumlah film: {num_movies}")

# Distribusi rating
print("\nDistribusi Rating:")
print(df['rating'].value_counts().sort_index())

# Film dan pengguna dengan sedikit rating
user_rating_count = df.groupby('user_id').size()
movie_rating_count = df.groupby('movie_id').size()

print(f"\nPengguna dengan <20 rating: {(user_rating_count < 20).sum()} pengguna")
print(f"Film dengan <20 rating: {(movie_rating_count < 20).sum()} film")

Jumlah pengguna: 943
Jumlah film: 1682

Distribusi Rating:
rating
1     6110
2    11370
3    27145
4    34174
5    21201
Name: count, dtype: int64

Pengguna dengan <20 rating: 0 pengguna
Film dengan <20 rating: 743 film


#3. Recomendation Method
- Menggunakan User-Based Collaborative Filtering dengan pendekatan cosine similarity antar pengguna.
- Membuat matriks rating pengguna dan menghitung kemiripan antar pengguna.
- Memberikan rekomendasi film untuk user ID 28 berdasarkan rating dari pengguna yang paling mirip dengannya.

## **User-Based Collaborative Filtering**

In [19]:
#Import Library
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [18]:
# Buat matrix rating
user_movie_matrix = df.pivot_table(index='user_id', columns='title', values='rating')

# Hitung similarity antar pengguna
user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

# Fungsi rekomendasi untuk user
def recommend_movies_user_based(user_id, n_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:6]
    user_ratings = user_movie_matrix.loc[user_id]
    weighted_scores = {}

    for other_user, similarity in similar_users.items():
        other_ratings = user_movie_matrix.loc[other_user]
        for movie in user_movie_matrix.columns:
            if pd.isna(user_ratings[movie]) and not pd.isna(other_ratings[movie]):
                if movie not in weighted_scores:
                    weighted_scores[movie] = 0
                weighted_scores[movie] += similarity * other_ratings[movie]

    recommendations = sorted(weighted_scores.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]
    return [movie for movie, score in recommendations]

Rekomendasi untuk user ID 28 (User-Based):
1. Alien (1979)
2. Natural Born Killers (1994)
3. From Dusk Till Dawn (1996)
4. Tales from the Crypt Presents: Bordello of Blood (1996)
5. Fifth Element, The (1997)


# 4. System Evaluation
- Menampilkan 5 rekomendasi film untuk user ID 28.
- Mengevaluasi hasil secara deskriptif dengan menganalisis genre film, pola kesamaan pengguna, dan kesesuaian rekomendasi dengan preferensi user yang mirip.

## Rekomendasi:

In [20]:
print("Rekomendasi untuk user ID 28 (User-Based):")
for i, title in enumerate(recommend_movies_user_based(28), start=1):
    print(f"{i}. {title}")

Rekomendasi untuk user ID 28 (User-Based):
1. Alien (1979)
2. Natural Born Killers (1994)
3. From Dusk Till Dawn (1996)
4. Tales from the Crypt Presents: Bordello of Blood (1996)
5. Fifth Element, The (1997)


## Evaluasi Deskriptif:

- Film yang direkomendasikan mayoritas bergenre sci-fi, horror, dan thriller, menunjukkan bahwa user ID 28 kemungkinan menyukai film dengan tema aksi, kekerasan, atau supranatural.

- Sistem menggunakan cosine similarity antar pengguna untuk menemukan pengguna mirip, lalu merekomendasikan film yang belum ditonton tapi disukai pengguna tersebut.

- Film seperti Alien dan The Fifth Element populer dan sering mendapat rating tinggi, mendukung kualitas rekomendasi.

- Namun, jika riwayat rating user 28 masih sedikit, rekomendasi bisa terlalu dipengaruhi satu pengguna mirip saja.